<a href="https://colab.research.google.com/github/nassim-fox/semi-supervised-ae/blob/master/semi_supervised_AE_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Reshape, Flatten
from keras.models import Model
from keras import backend as K
import tensorflow as tf
import numpy as np


# ae

In [0]:

latent_dim = 32 

#encoder
input_e = Input(shape=(28, 28, 1))  

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_e)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
s_shape = K.int_shape(x)
x = Flatten()(x)
x = Dense(512,activation="relu")(x)
latent = Dense(latent_dim)(x)

encoder = Model(input_e,latent,name="encoder")

In [0]:

#decoder

input_d = Input(shape=(latent_dim,))
x = Dense(np.prod(s_shape[1:]))(input_d)
x = Reshape((s_shape[1],s_shape[2],s_shape[3]))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
output = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

decoder = Model(input_d,output,name="decoder")
decoder.summary()

#not needed
autoencoder = Model(input_e,decoder(encoder(input_e)))
autoencoder.compile(optimizer='adam', loss='binary_crossentropy') 


# cnn

In [15]:

input_c = Input(shape=(latent_dim,))
x = Dense(np.prod(s_shape[1:]))(input_c)
x = Reshape((s_shape[1],s_shape[2],s_shape[3]))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Flatten()(x)
x = Dense(128,activation="relu")(x)
x = Dense(10,activation="softmax")(x)

cnn = Model(input_c,x)

cnn.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 32)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 128)               4224      
_________________________________________________________________
reshape_7 (Reshape)          (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 4, 4, 32)          2336      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 2, 2, 32)          0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 2, 2, 16)          4624      
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 1, 1, 16)          0   

In [17]:
model = Model(input_e,[cnn(encoder(input_e)),decoder(encoder(input_e))])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
encoder (Model)                 (None, 32)           84368       input_13[0][0]                   
                                                                 input_13[0][0]                   
__________________________________________________________________________________________________
model_2 (Model)                 (None, 10)           14650       encoder[3][0]                    
__________________________________________________________________________________________________
decoder (Model)                 (None, 28, 28, 1)    6705        encoder[4][0]              

# train

In [0]:
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1)) 
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [25]:

model.compile(optimizer="adam",loss=["categorical_crossentropy","mse"],metrics=["accuracy"])

model.fit(x_train, [y_train,x_train],
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, [y_test,x_test]))


Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 13s 209us/step - loss: 0.7020 - model_2_loss: 0.6445 - decoder_loss: 0.0573 - model_2_accuracy: 0.7756 - decoder_accuracy: 0.7963 - val_loss: 0.2753 - val_model_2_loss: 0.2331 - val_decoder_loss: 0.0404 - val_model_2_accuracy: 0.9276 - val_decoder_accuracy: 0.7990
Epoch 2/50
60000/60000 [==============================] - 6s 94us/step - loss: 0.1991 - model_2_loss: 0.1630 - decoder_loss: 0.0361 - model_2_accuracy: 0.9492 - decoder_accuracy: 0.7990 - val_loss: 0.1712 - val_model_2_loss: 0.1360 - val_decoder_loss: 0.0341 - val_model_2_accuracy: 0.9555 - val_decoder_accuracy: 0.8002
Epoch 3/50
60000/60000 [==============================] - 6s 95us/step - loss: 0.1416 - model_2_loss: 0.1101 - decoder_loss: 0.0314 - model_2_accuracy: 0.9661 - decoder_accuracy: 0.8022 - val_loss: 0.1143 - val_model_2_loss: 0.0844 - val_decoder_loss: 0.0290 - val_model_2_accuracy: 0.9726 - val_decoder_ac